## Snakemake Demo


Modified from `https://github.com/ctb/2019-snakemake-ucdavis`


### Tasks we are going to put in our workflow

Inside the `data` folder, there are 4 fastq.gz files. We will use `Snakemake` to run `fastqc` on each fastq files and compile multiple html files into a single one using `multiQC`.

(You can think of these files as maybe input from a high-throughput sequencing session where we want to do some quality analysis.)

## Software we need to use

- Snakemake

- Bash (or any shell that will run Snakemake)

- Text editor to edit the `Snakefile` 

In this example, we'll also use the `fastqc` and `multiqc` programs.


#### SAMPLE DATA

In [ ]:
!curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/0Hour_001_1.fq.gz > data/0Hour_001_1.fq.gz
!curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/6Hour_001_1.fq.gz > data/6Hour_001_1.fq.gz
!curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/0Hour_001_2.fq.gz > data/0Hour_001_2.fq.gz
!curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/6Hour_001_2.fq.gz > data/6Hour_001_2.fq.gz

#### RESET DATA

In [28]:
!rm data/*.html 
!rm data/*.zip
!rm multiqc_data/ -r 
!rm *.html

!echo "# My snakefile\n# Will run by default when we type 'snakemake'\n\n" > Snakefile

## Running snakemake

### First Snakefile

In [ ]:
rule run_fastqc:
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

Copy-paste into Snakefile.

Save and run (using bash)

*NOTE: Snakemake by default runs the file in the directory called 'snakefile'*

In [ ]:
!snakemake --cores 1

Now we have a new `.html` file that was generated by `fastqc`. 

Points to make:
* the snakemake configuration file is by default called `Snakefile`

### Updating the Snakefile to track inputs and outputs

At the moment this is basically just a shell script with extra syntax.  Where is the value?

Well, shell scripts will rerun the command every time you run the file, even if there's no reason to do so because the file hasn't changed. 

With snakemake, you can annotate the rule with input and output files!

**NOTE:** This is particularly important for large or long workflows, where you're dealing with 10s to 100s of files that may take hours to days to process! It can be hard to figure out which files to rerun, but snakemake can really help you do this!

Updating our snakefile...

In [ ]:
rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

here, we've annotated the rule with the required
**input** file, as well as the expected **output** files.


Now run:

In [ ]:
!snakemake --cores 1

and you should see:
```
Building DAG of jobs...
Nothing to be done.
Complete log: .snakemake/log/2019-02-27T132031.813143.snakemake.log
```

Why? 

snakemake looked at the file, saw that the output files existed, and figured out that it didn't need to do anything.

### Forcibly re-running things

You can tell snakemake to run all the rules no matter what with `-F`:

In [ ]:
!snakemake -F --cores 1

Equivalently, you can also remove an output file and it will automatically re-run:

In [ ]:
rm data/*.html
snakemake

note that you don't need to remove *all* the output files to rerun a command - just remove *one* of them.

You can *also* update the timestamp on an *input* file, and snakemake will figure out that the output file is older than the input file, and rerun things.

In [ ]:
touch data/*.fq.gz
snakemake

Another useful snakemake option is the `--dryrun` option, this will show you what snakemake would do but won't actually run any of the rules.  This is useful to check that snakemake will do the jobs that you expect will be executed (without actually waiting for them to run, especially if they are computationally intensive).

In [ ]:
snakemake --dryrun

### Multiple rules

Let's add a rule to run fastqc on a second file:

In [ ]:
rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/6Hour_001_1.fq.gz"

In [ ]:
!snakemake --cores 1

Now, if you run this, what we might expect won't happen: snakemake will do nothing.  Why?

Well, snakemake only runs the *first* rule in a Snakefile, by default. You can give a rule name on the command line, if you like, **or** you can tell snakemake what output file(s) you want. 

Note that the first rule is not dependent on the execution of the second rule, i.e. snakemake can fulfill the input / outputs desired from the first rule alone; and thus it ignores the second rule unless we specify that it should handle it.

But the conventional way to do this in snakemake is to add an "all" rule as the first rule, let's try it. 


### A first refactoring: adding a better default rule

Let's start refactoring (cleaning up) this Snakefile.

First, let's add a rule at the top:

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/6Hour_001_1.fq.gz"

In [ ]:
!snakemake --cores 1

This is a blank rule that gathers together all of the various files you want produced, and says "hey, snakemake, I depend on all of these files for my input - make them for me!"

Let's run snakemake again and now you should see the second fastqc command run, with the appropriate output files!

Note that snakemake only runs the second rule, because it looks at the output files and sees that the first file you wanted, `0Hour_001_1_fastqc.html` already exists!

### A second refactoring: doing a bit of templating

There's a lot of repetition in each of these rules. Let's collapse it down a little bit by replacing the filename in the fastqc command with a magic variable, `{input}`.

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc {input}"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc {input}"

This all works as before, but now the rule is a bit more generic.

### Refactoring 3: templating output files, too

The output filenames ALSO depend on the input file names in some way - specifically, fastqc replace part of the filename with `_fastqc.html` and `_fastqc.zip` to make its two output files.

Let's rewrite the rule using some snakemake pattern matching:

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

rule fastqc_a_file2:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

What we've done here is tell snakemake that anytime we say we *want* a file that ends with `_fastqc.html`, it should look for a file that ends in `.fq.gz` and then run `fastqc` on it.

If you look at the rules above, note that they are now identical rules (since we made them more generic with `{input}` and `{filename}` wildcards (which could be anything)) 

Let's remove one of the rules to collapse down our snakefile:

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

Note that the variable name in input and output does not have to be "filename", it can be anything (that matches the pattern) as long as you're consistant.  So let's run snakemake again.

In [ ]:
!rm data/*.html
!snakemake --cores 1

### Building out the workflow

So, we've gotten our snakefile to run `fastqc` on several files in a generic (reusable!) way.

Let's add in a new rule - multiqc, to summarize our fastqc intermediate results.

multiqc takes a directory name under which there are one or more fastqc reports, and then summarizes them.

Running it on the command line,

In [ ]:
!multiqc data

This will create two outputs, `multiqc_report.html` and the directory `multiqc_data/` which contains a bunch of files. Let's create a snakemake rule for this; add:

In [ ]:
fastqc_output = ["data/0Hour_001_1_fastqc.html", "data/6Hour_001_1_fastqc.html",
  "data/0Hour_001_2_fastqc.html", "data/6Hour_001_2_fastqc.html"]

rule all:
  input:
    "multiqc_report.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

rule run_multiqc:
  input:
    fastqc_output
  output:
    "multiqc_report.html",
    directory("multiqc_data")
  shell:
    "multiqc data/"

In [27]:
!rm multiqc_report.html
!snakemake --cores 1

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job            count    min threads    max threads
-----------  -------  -------------  -------------
all                1              1              1
run_multiqc        1              1              1
total              2              1              1

Select jobs to execute...

[Thu Nov 18 12:57:55 2021]
rule run_multiqc:
    input: data/0Hour_001_1_fastqc.html, data/6Hour_001_1_fastqc.html, data/0Hour_001_2_fastqc.html, data/6Hour_001_2_fastqc.html
    output: multiqc_report.html, multiqc_data
    jobid: 1
    resources: tmpdir=/tmp


  /// ]8;id=863756;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.11

|           multiqc | Search path : /home/cam/SNAKEMAKE/DEMO/data
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 12/12  
|            fastqc | Found 4 reports
|           multiqc | Compressing plot da

As you can see, this is an elegant way of keeping track of dependencies, and ordering jobs accordingly.



Points to make:

* other than the first rule, rules can be in any order
* the rule name doesn't really matter, it's mostly for debugging. It just needs to be "boring" (text, underscores, etc. only)


We can fix the first issue by using **variables**. 

To use variables, let's make a Python list at the very top, containing all of our expected output files from fastqc:

In [ ]:
fastqc_output = ["data/0Hour_001_1_fastqc.html", "data/6Hour_001_1_fastqc.html",
  "data/0Hour_001_2_fastqc.html", "data/6Hour_001_2_fastqc.html"]

### Thanks for listening! 


-- Jayden, Milly, James, Cam 